In [1]:
pip install torch transformers pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

In [3]:
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p2')
model = BertModel.from_pretrained('indobenchmark/indobert-base-p2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

In [4]:
def get_text_embedding(text, model, tokenizer, max_length=256):
    if pd.isna(text):
        return np.zeros(256)  # Jika teks kosong, kembalikan array nol

    # Tokenisasi teks
    inputs = tokenizer(text, return_tensors="pt", max_length=max_length, truncation=True, padding="max_length")

    # Ekstraksi embedding menggunakan model IndoBERT
    with torch.no_grad():
        outputs = model(**inputs)

    # Mengambil vektor embedding dari hidden state terakhir (layer terakhir)
    last_hidden_state = outputs.last_hidden_state  # Shape: [batch_size, seq_len, hidden_size]

    # Rata-rata sepanjang dimensi sequence (dim=1) untuk menghasilkan vektor dengan ukuran [hidden_size]
    embedding = last_hidden_state.mean(dim=1).squeeze().numpy()

    # Jika hasil embedding lebih besar dari 256, potong ke 256 dimensi
    return embedding[:256]

In [5]:
df = pd.read_csv("/content/preprocessed_output.csv")

In [6]:
df['nama_embedding'] = df['processed_nama'].apply(lambda x: get_text_embedding(x, model, tokenizer).tolist())
df['deskripsi_embedding'] = df['processed_deskripsi'].apply(lambda x: get_text_embedding(x, model, tokenizer).tolist())


In [7]:
df.to_json("text_embeddings_output.json", orient="records", lines=True)
